# Imports

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import pandas as pd
import keras
from keras.utils import FeatureSpace

# Constants

In [ ]:
PLAYER_ROLES = [f"radiant_player_{x}_role" for x in range(1, 6)]

['radiant_player_1_role',
 'radiant_player_2_role',
 'radiant_player_3_role',
 'radiant_player_4_role',
 'radiant_player_5_role']

# Fetch Data

In [ ]:
df = pd.read_parquet("dota2_matches.parquet")
df.head()

,league,league_id,league_tier,league_start_date_time,league_end_date_time,league_region,series_id,series_type,match_id,match_start_date_time,...,dire_player_5_hero_id,dire_player_5_hero,dire_player_5_position,dire_player_5_lane,dire_player_5_role,dire_player_5_kills,dire_player_5_deaths,dire_player_5_assists,dire_player_5_networth,game_version_id
0,Dota 2 Space League,17163,PROFESSIONAL,2024-10-12 21:00:00,2025-03-13 22:00:00,<NA>,915196,BEST_OF_THREE,7988517842,2024-10-15 01:28:24,...,108,Underlord,POSITION_3,OFF_LANE,CORE,8,8,12,19449,177
1,Royal Circuit #3,17064,PROFESSIONAL,2024-10-01 05:00:00,2024-10-31 05:00:00,<NA>,915178,BEST_OF_THREE,7988515816,2024-10-15 01:24:03,...,20,Vengeful Spirit,POSITION_5,SAFE_LANE,HARD_SUPPORT,1,7,13,7981,177
2,Dota 2 Space League,17163,PROFESSIONAL,2024-10-12 21:00:00,2025-03-13 22:00:00,<NA>,915196,BEST_OF_THREE,7988495156,2024-10-15 00:43:58,...,14,Pudge,POSITION_5,SAFE_LANE,HARD_SUPPORT,2,4,10,10220,177
3,Royal Circuit #3,17064,PROFESSIONAL,2024-10-01 05:00:00,2024-10-31 05:00:00,<NA>,915178,BEST_OF_THREE,7988474098,2024-10-15 00:01:33,...,83,Treant Protector,POSITION_5,SAFE_LANE,HARD_SUPPORT,3,2,29,9250,177
4,Dota 2 Space League,17163,PROFESSIONAL,2024-10-12 21:00:00,2025-03-13 22:00:00,<NA>,915166,BEST_OF_THREE,7988466451,2024-10-14 23:44:45,...,138,Muerta,POSITION_4,OFF_LANE,LIGHT_SUPPORT,6,5,13,15004,177


# EDA

In [7]:
df.nunique().sort_values(ascending=True)

series_type                   4
radiant_player_1_role         5
radiant_player_3_role         5
radiant_player_2_role         5
radiant_player_4_role         5
                          ...  
radiant_team_id           36037
dire_team_id              37199
match_start_date_time     38001
series_id                117529
match_id                 193466
Length: 130, dtype: int64

In [ ]:
for role in PLAYER_ROLES:
  df[role] = df[role].str.strip().replace("", pd.NA).replace("UNKNOWN", pd.NA)
  df = df.dropna(subset=[role])

df.reset_index(drop=True, inplace=True)
df[PLAYER_ROLES]

,radiant_player_1_role,radiant_player_2_role,radiant_player_3_role,radiant_player_4_role,radiant_player_5_role
0,CORE,CORE,LIGHT_SUPPORT,CORE,HARD_SUPPORT
1,LIGHT_SUPPORT,CORE,CORE,CORE,HARD_SUPPORT
2,CORE,LIGHT_SUPPORT,CORE,HARD_SUPPORT,CORE
3,LIGHT_SUPPORT,HARD_SUPPORT,CORE,CORE,CORE
4,CORE,HARD_SUPPORT,LIGHT_SUPPORT,CORE,CORE
...,...,...,...,...,...
155319,CORE,CORE,CORE,CORE,CORE
155320,CORE,CORE,CORE,CORE,CORE
155321,CORE,CORE,CORE,CORE,CORE
155322,CORE,CORE,CORE,CORE,CORE
